In [52]:
import sys 
import os 
sys.path.insert(0, os.path.dirname(os.getcwd()))

from utils import get_connection
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pickle 
from tqdm import tqdm 
import numpy as np

import tensorflow as tf

In [11]:
def load_model_and_encoder(model_path):
    model_path = model_path
    model = tf.keras.models.load_model(model_path)

    with open(f'{model_path}-label_encoder.pkl'.replace("_final", ''), 'rb') as f :
        label_encoder = pickle.load(f)
    
    return model, label_encoder

In [20]:
DATAPATH = '/root/autodl-tmp'

MODEL_MAP = {
    "occasion_cn": os.path.join(DATAPATH, "occasion_cn/20240625153648"),
    "needs_cn": os.path.join(DATAPATH, "needs_cn/20240626012224_final"),
    "where_en": os.path.join(DATAPATH, "where_en/20240626012639_final"),
    "who_cn": os.path.join(DATAPATH, "who_cn/20240626023357_final"),
}


occasion_cn_model, occasion_cn_encoder = load_model_and_encoder(MODEL_MAP['occasion_cn'])
needs_cn_model, needs_cn_encoder = load_model_and_encoder(MODEL_MAP['needs_cn'])
where_en_model, where_en_encoder = load_model_and_encoder(MODEL_MAP['where_en'])
who_cn_model, who_cn_encoder = load_model_and_encoder(MODEL_MAP['who_cn'])

occasion_cn_model.summary()
print(occasion_cn_encoder.categories_[0].__len__(), occasion_cn_encoder.categories_)

needs_cn_model.summary()
print(needs_cn_encoder.categories_[0].__len__(), needs_cn_encoder.categories_)

where_en_model.summary()
print(where_en_encoder.categories_[0].__len__(), where_en_encoder.categories_)

who_cn_model.summary()
print(who_cn_encoder.categories_[0].__len__(), who_cn_encoder.categories_)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ffn1 (Dense)                (None, 896)               803712    
                                                                 
 dropout_1 (Dropout)         (None, 896)               0         
                                                                 
 layer_normalization (LayerN  (None, 896)              1792      
 ormalization)                                                   
                                                                 
 classification_header (Dens  (None, 151)              135447    
 e)                                                              
                                                                 
Total params: 940,951
Trainable params: 940,951
Non-trainable params: 0
_________________________________________________________________
151 [array(['Others', '一个人创业', '一个人的屏幕时光', '一人食光

In [105]:
OUPUT_PATH = os.path.join(DATAPATH, '20m_tag')

os.makedirs(OUPUT_PATH, exist_ok = True)

exist_rst = os.listdir(OUPUT_PATH)

embeddings_path = os.path.join(DATAPATH, 'qwen2_05B_embeddings')
embedding_file_ls = os.listdir(embeddings_path)

embedding_file_ls = set(embedding_file_ls).difference(exist_rst)

embedding_file_ls = list(map(lambda x: os.path.join(embeddings_path, x),    embedding_file_ls))

embedding_file_ls = list(filter(lambda x: os.path.getsize(x) >0, embedding_file_ls))

for embed_path in tqdm(embedding_file_ls):
    with open(embed_path, 'rb') as f:
        df_embed = pickle.load(f)

    y_occasion_cn = occasion_cn_encoder.inverse_transform(occasion_cn_model.predict(exbedding_X, verbose=0)).reshape(1,-1)[0]
    y_needs_cn = needs_cn_encoder.inverse_transform(needs_cn_model.predict(exbedding_X, verbose=0)).reshape(1,-1)[0]
    y_where_en = where_en_encoder.inverse_transform(where_en_model.predict(exbedding_X, verbose=0)).reshape(1,-1)[0]
    y_who_cn = who_cn_encoder.inverse_transform(who_cn_model.predict(exbedding_X, verbose=0)).reshape(1,-1)[0]

    df_rst = pd.DataFrame({'id': idx, 'occasion_cn': y_occasion_cn, 'where_en': y_where_en, 'need_cn': y_needs_cn, 'who_cn': y_who_cn})


    with open(embed_path.replace(embeddings_path, OUPUT_PATH), 'wb') as f:
        pickle.dump(df_rst, f)

100%|██████████| 1984/1984 [2:04:16<00:00,  3.76s/it]  


In [106]:



df_rst

,id,occasion_cn,where_en,need_cn,who_cn
0,e1da7f2bef1fffe75f344480468ad791|8910754ee9607...,家庭庆祝,Hotel,穷乐/平替,商务人士
1,b663da3c8019d1bf17301ae0b3dd03a9|9bcf48206501c...,家庭庆祝,Home,与他人分享喜悦,其他
2,ddfd0c0003f39aed1378faa4ff2d72cd|448634b6d1f35...,家庭聚会,Home,为了充饥,家人
3,7239cab97c70951e56ea5013a25eae3a|3cb1f463fd81c...,一人食光,Home,减脂期解馋,其他
4,003d00f409b85f900df0ca88bdd1177f|3a5bb6dcc84ce...,家庭聚会,Home,满足口腹欲,其他
...,...,...,...,...,...
9995,7ce7e47bae28e60a2fba80e50f134abc|a304c057a6f25...,家庭庆祝,Home,健康养生,其他
9996,d7180379712c4bf10178aae9c7f1601d|e575e5015beb1...,办公室派对,Office,享受口感,其他
9997,8b67e6887e9d5cde54c824c5963e39ec|7a142699e1b29...,野餐时（露营）-与朋友,Others,Others,其他
9998,732d0789949f0f9ea7d67d928bfdc188|7039f8e85a5e9...,Others,Others,体验新奇,Others
